In [1]:
import dask.dataframe as dd
# this should whatever your locomizer data file is
#%time temp = dd.read_csv("../data/sample/signals_set_2.csv")
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow.feather as ft

In [3]:
ipc = dd.read_csv('PATENTS/Patents/202001_EPO_IPC.txt', delimiter='|')
ipc.head()

,appln_id,prio_year,app_year,IPC
0,1,1999,2000,G06K007/00
1,1,1999,2000,G06K017/00
2,1,1999,2000,G06K019/077
3,1,1999,2000,H01R012/18
4,1,1999,2000,H04M001/02


In [17]:
inv = pd.read_csv('data/INV_IPC_CLASS.csv')
inv.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06
1,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61
2,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07
3,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C12
4,EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingat...",UKH33,GB,1.0,1.0,1978.0,G03


In [9]:
prior = ipc[['prio_year', 'appln_id']].compute()

In [10]:
prior.head()

,prio_year,appln_id
0,1999,1
1,1999,1
2,1999,1
3,1999,1
4,1999,1


In [18]:
len(inv)

304237

In [19]:
inv_prior = inv.merge(prior, on='appln_id', how='left')

In [20]:
len(inv_prior)

1596112

In [21]:
inv_prior.head(10)

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC,prio_year
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06,1977
1,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06,1977
2,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06,1977
3,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61,1977
4,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61,1977
5,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61,1977
6,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07,1977
7,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07,1977
8,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07,1977
9,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C12,1977


In [22]:
inv_prior = inv_prior.drop_duplicates()

In [23]:
len(inv_prior)

304237

In [24]:
inv_prior.to_csv('data/INV_IPC_CLASS_prior.csv', index=False)